In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense, RepeatVector, TimeDistributed


import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('TurinAHU.csv')
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Extracting day, month, year, and time into separate columns
df["day"] = df["Timestamp"].dt.day
df["month"] = df["Timestamp"].dt.month
df["year"] = df["Timestamp"].dt.year
df["time"] = df["Timestamp"].dt.hour * 3600 + df["Timestamp"].dt.minute * 60 + df["Timestamp"].dt.second

# Dropping the "timestamp" column
df = df.drop("Timestamp", axis=1)

# Reordering the columns
cols = df.columns.tolist()
cols = ["time", "day", "month", "year"] + cols[:-4]
df = df[cols]
df

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power
0,19.859999,20.469999,18.5,19.020000,20.299999,71.110001,58.919998,79.5,0.0,0.0
1,19.855000,20.430000,18.5,19.020000,20.299999,71.320000,59.000000,82.0,0.0,0.0
2,19.850000,20.410000,18.5,19.020000,20.299999,71.470001,59.109997,79.5,0.0,0.0
3,19.840000,20.379999,18.5,19.080000,20.299999,71.439995,59.309998,77.0,0.0,0.0
4,19.830000,20.350000,18.5,19.080000,20.299999,71.580002,59.559998,79.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
33883,19.539999,20.004999,20.5,19.619999,14.700000,39.020000,27.930000,57.0,0.0,0.0
33884,19.520000,19.949999,20.5,19.539999,13.700000,39.020000,28.090000,57.0,0.0,0.0
33885,19.430000,19.955000,20.5,19.420000,13.700000,39.399998,27.930000,57.0,0.0,0.0
33886,19.420000,19.920000,20.5,19.400000,13.700000,39.599998,28.039999,57.0,0.0,0.0


In [9]:
############################################################# STATELESS GRU #############################################################

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# Prepare the data for training
X = []
y = []
n_future = 2 # number of timesteps to predict
n_past = 96 # number of timesteps to use as input
for i in range(n_past, len(data_scaled) - n_future + 1):
    X.append(data_scaled[i - n_past:i, :])
    y.append(data_scaled[i:i + n_future, :])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model architecture
model = Sequential()
model.add(GRU(32, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
#model.add(GRU(64, activation='tanh', dropout=0.5, return_sequences=True))
model.add(GRU(32, activation='tanh', dropout=0.5))
model.add(RepeatVector(n_future))
#model.add(GRU(64, activation='tanh', dropout=0.5, return_sequences=True))
#model.add(GRU(64, activation='tanh', dropout=0.5, return_sequences=True))
model.add(TimeDistributed(Dense(X_train.shape[2], activation='linear')))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2]))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1], y_test.shape[2]))

mae = np.mean(np.abs(y_pred_rescaled - y_test_rescaled), axis=0)
mape = np.mean(np.abs((y_pred_rescaled - y_test_rescaled) / y_test_rescaled), axis=0) * 100
rmse = np.sqrt(np.mean(np.square(y_pred_rescaled - y_test_rescaled), axis=0))

print('MAE:', mae)
print('MAPE:', mape)
print('RMSE:', rmse)

Epoch 1/100
296/296 [==============================] - 25s 70ms/step - loss: 0.0302 - val_loss: 0.0079
Epoch 2/100
296/296 [==============================] - 20s 68ms/step - loss: 0.0095 - val_loss: 0.0057
Epoch 3/100
296/296 [==============================] - 20s 69ms/step - loss: 0.0075 - val_loss: 0.0050
Epoch 4/100
296/296 [==============================] - 20s 68ms/step - loss: 0.0067 - val_loss: 0.0046
Epoch 5/100
296/296 [==============================] - 17s 59ms/step - loss: 0.0063 - val_loss: 0.0044
Epoch 6/100
296/296 [==============================] - 18s 61ms/step - loss: 0.0059 - val_loss: 0.0043
Epoch 7/100
296/296 [==============================] - 17s 58ms/step - loss: 0.0058 - val_loss: 0.0042
Epoch 8/100
296/296 [==============================] - 18s 59ms/step - loss: 0.0056 - val_loss: 0.0044
Epoch 9/100
296/296 [==============================] - 18s 61ms/step - loss: 0.0054 - val_loss: 0.0040
Epoch 10/100
296/296 [==============================] - 18s 61ms/step - l

KeyboardInterrupt: 